In [ ]:
# TensorFlow Lite Image Classification Model

In [1]:
!pip install tensorflow tensorflow-hub -qqq

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load MobileNetV2 model
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

# Create a Keras layer from the TensorFlow Hub model
keras_layer = hub.KerasLayer(model_url)

# Build the Sequential model with the Keras layer
model = tf.keras.Sequential([keras_layer])

# Check model summary
model.build([None, 224, 224, 3])
model.summary()

In [4]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Download and process a sample image
!wget -q https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -xf flower_photos.tgz

# Load and resize an image
img_path = "flower_photos/roses/12240303_80d87f77a3_n.jpg"
img = Image.open(img_path).resize((224, 224))
img_array = np.array(img) / 255.0
img_array = img_array[np.newaxis, ...]

# Predict using the model
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
print(f"Predicted class: {predicted_class}")

# Display the image
plt.imshow(img)
plt.title(f"Predicted class: {predicted_class}")
plt.axis('off')
plt.show()

NameError: name 'model' is not defined

In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to file
with open("mobilenet_v2.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as mobilenet_v2.tflite")

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="mobilenet_v2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and resize an image
img_path = "flower_photos/roses/12240303_80d87f77a3_n.jpg"
img = Image.open(img_path).resize((224, 224))

# Convert image to array and normalize
img_array = np.array(img) / 255.0
img_array = img_array.astype(np.float32)  # Convert to float32
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Run inference
interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

predicted_class = np.argmax(output_data[0])
print(f"Predicted class (TFLite): {predicted_class}")

In [ ]:
import os
import tensorflow as tf

# Save the Keras model to disk
model.save("mobilenet_v2.h5")

# Calculate the model size before conversion
model_size_bytes = os.path.getsize("mobilenet_v2.h5")

# Calculate the TFLite model size
tflite_model_size_bytes = os.path.getsize("mobilenet_v2.tflite")

# Convert the sizes to MB
model_size_mb = model_size_bytes / (1024 * 1024)
tflite_model_size_mb = tflite_model_size_bytes / (1024 * 1024)

print(f"Original Model Size: {model_size_mb:.2f} MB")
print(f"TFLite Model Size: {tflite_model_size_mb:.2f} MB")

In [ ]:
# Download ImageNet labels
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt
with open("ImageNetLabels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="mobilenet_v2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load ImageNet labels
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt
with open("ImageNetLabels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]

# Function to run inference, print the result, and show the image
def classify_image(image_path):
    img = Image.open(image_path).resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = img_array.astype(np.float32)
    img_array = np.expand_dims(img_array, axis=0)

    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    predicted_class = np.argmax(output_data[0])
    predicted_label = labels[predicted_class] if predicted_class < len(labels) else "Unknown"
    print(f"Predicted class: {predicted_label}")

    plt.imshow(img)
    plt.title(f"Predicted label: {predicted_label}")
    plt.axis('off')
    plt.show()

# Test with other images and show the results
classify_image("flower_photos/daisy/21652746_cc379e0eea_m.jpg")
# classify_image("flower_photos/sunflowers/6953297_8576bf4ea3.jpg")